Welcome, to Punisher. After running your strategy, view the live results with Dash

```
$ python -m punisher.charts.dash_charts.dash_record --name default_backtest
```

## Define Strategy

In [337]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from punisher.common import *

from punisher.strategies.strategy import Strategy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [335]:
df = pd.DataFrame()

In [336]:
df.copy()

""


In [338]:
class SimpleStrategy(Strategy):
    def __init__(self, asset, quantity):
        super().__init__()
        self.asset = asset
        self.quantity = quantity

    def log_all(self, orders, data, ctx, time_utc):
        self.logger = ctx.logger
        if self.logger is not None:
            self.log_epoch_time(time_utc)
            self.log_ohlcv(data)
            self.log_orders(orders)
            self.log_performance(ctx)
            self.log_balance(ctx)
            self.log_positions(ctx)
            self.log_metrics(ctx)

    def handle_data(self, data, ctx):
        orders = []
        price = data.get('close')
        quantity = .05

        if random.random() > 0.5:
            order = order_manager.build_limit_buy_order(
                ctx.exchange, self.asset, quantity, price)
            orders.append(order)
        else:
            order = order_manager.build_market_sell_order(
                ctx.exchange, self.asset, quantity)
            orders.append(order)

        # Optionally cancel pending orders (LIVE trading)
        #pending_orders = ctx.exchange.fetch_open_orders(asset)
        cancel_ids = []

        # Add Metrics and OHLCV to Record
        self.update_metric('SMA', 5.0, ctx)
        self.update_metric('RSI', 10.0, ctx)
        self.update_ohlcv(data, ctx)
        print(data.get('time_utc'))
        self.log_all(orders, data, ctx, data.get('time_utc'))
        return {
            'orders': orders,
            'cancel_ids': cancel_ids
        }

## Backtest

In [345]:
cash = 1.0
ohlcv_fpath = os.path.join(cfg.DATA_DIR, c.DEFAULT_30M_FEED_CSV_FILENAME)
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.THIRTY_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
feed = OHLCVFileFeed(
    fpath=ohlcv_fpath,
    start=None, # Usually None for backtest, but its possible to filter the csv
    end=None
)
exchange = load_feed_based_paper_exchange(balance, feed)
asset = Asset(c.ETH, c.BTC)
strategy = SimpleStrategy(asset, quantity=.05)
experiment_name = 'default'

In [346]:
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

---------------------------------------
Epoch 1 - Timestep: 2018-01-09T00:00:00
---------------------------------------
OHLCV
    O: 0.0760 | C: 0.0766 | V: 7900.8 | T: 2018-01-09T00:00:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0766 | Quantity: 0.0500
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 0.0}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 2 - Timestep: 2018-01-09T00:30:00
---------------------------------------
OHLCV
    O: 0.0766 | C: 0.0774 | V: 7715.3 | T: 2018-01-09T00:30:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0774 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9962 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.99616755, 'used': 0.0, 'total': 0.99616755}
    ETH - {'free': 0.05, 'used': 0.0, 'total': 0.05}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.05, 'cost_price': 0.076649, 'latest_price': 0.076649

2018-01-09 00:00:00
Equal? False
2018-01-09 00:30:00
Equal? False
2018-01-09 01:00:00
Equal? False
2018-01-09 01:30:00
2018-01-09 01:30:00
Equal? False
2018-01-09 02:00:00
Equal? False
2018-01-09 02:30:00
Equal? False
2018-01-09 03:00:00
Equal? False


---------------------------------------
Epoch 8 - Timestep: 2018-01-09T03:30:00
---------------------------------------
OHLCV
    O: 0.0785 | C: 0.0791 | V: 10039.6 | T: 2018-01-09T03:30:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0791 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9806 Total Val: 1.0002 PnL: 0.0001 Returns: 0.0001
BALANCE
    BTC - {'free': 0.98058265, 'used': 0.0, 'total': 0.98058265}
    ETH - {'free': 0.25, 'used': 0.0, 'total': 0.25}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.25, 'cost_price': 0.07755186666666666, 'latest_price': 0.078481}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 9 - Timestep: 2018-01-09T04:00:00
---------------------------------------
OHLCV
    O: 0.0791 | C: 0.0790 | V: 9275.8 | T: 2018-01-09T04:00:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9766 Total Val: 1.0004 PnL: 0.0002 Returns: 0.0002
BALANCE
    BTC - {'free': 0.97662715, 'used': 0.0, '

2018-01-09 03:30:00
Equal? False
2018-01-09 04:00:00
2018-01-09 04:00:00
Equal? False
2018-01-09 04:30:00
2018-01-09 04:30:00
Equal? False
2018-01-09 05:00:00
Equal? False
2018-01-09 05:30:00
2018-01-09 05:30:00
Equal? False
2018-01-09 06:00:00


METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 14 - Timestep: 2018-01-09T06:30:00
---------------------------------------
OHLCV
    O: 0.0800 | C: 0.0803 | V: 8760.2 | T: 2018-01-09T06:30:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9805 Total Val: 1.0005 PnL: 0.0004 Returns: 0.0004
BALANCE
    BTC - {'free': 0.9805333500000001, 'used': 0.0, 'total': 0.9805333500000001}
    ETH - {'free': 0.25, 'used': 0.0, 'total': 0.25}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.25, 'cost_price': 0.07852163555555557, 'latest_price': 0.07998}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 15 - Timestep: 2018-01-09T07:00:00
---------------------------------------
OHLCV
    O: 0.0803 | C: 0.0800 | V: 7891.7 | T: 2018-01-09T07:00:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9845 Total Val: 1.0006 PnL: 0.0005 Returns: 0.000

...

2018-01-11 19:00:00
2018-01-11 19:00:00
Equal? False
Equal? False
2018-01-11 19:30:00
2018-01-11 19:30:00
Equal? False
2018-01-11 20:00:00
2018-01-11 20:00:00
Order Failed {
    "id": "da20d5b9ee0148d684adc740ff00ed46",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.05,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "FAILED",
    "created_time": "2018-01-19T20:32:39.453173",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "attempts": 0,
    "trades": [],
    "error": {
        "code": "INSUFFICIENT_FUNDS",
        "msg": "Insufficient funds to place order for asset ETH/BTC of cost: 0.00439. BTC balance is: 1.0020653499999996. ETH balance is 1.3877787807814457e-17"
    },
    "symbol": "ETH/BTC"
}
2018-01-11 20:30:00


---------------------------------------
Epoch 139 - Timestep: 2018-01-11T21:00:00
---------------------------------------
OHLCV
    O: 0.0882 | C: 0.0878 | V: 4022.5 | T: 2018-01-11T21:00:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0878 | Quantity: 0.0500
PERFORMANCE
    Cash: 1.0021 Total Val: 1.0021 PnL: 0.0021 Returns: 0.0021
BALANCE
    BTC - {'free': 1.0020653499999996, 'used': 0.0, 'total': 1.0020653499999996}
    ETH - {'free': 1.3877787807814457e-17, 'used': 0.0, 'total': 1.3877787807814457e-17}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 1.3877787807814457e-17, 'cost_price': 0.08766799999999998, 'latest_price': 0.088062}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 140 - Timestep: 2018-01-11T21:30:00
---------------------------------------
OHLCV
    O: 0.0879 | C: 0.0876 | V: 4851.5 | T: 2018-01-11T21:30:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9977 Total Val: 1.0021 PnL: 0.0

2018-01-11 20:30:00
Order Failed {
    "id": "da20d5b9ee0148d684adc740ff00ed46",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.05,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "FAILED",
    "created_time": "2018-01-19T20:32:39.453173",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "attempts": 1,
    "trades": [],
    "error": {
        "code": "INSUFFICIENT_FUNDS",
        "msg": "Insufficient funds to place order for asset ETH/BTC of cost: 0.00440. BTC balance is: 1.0020653499999996. ETH balance is 1.3877787807814457e-17"
    },
    "symbol": "ETH/BTC"
}
2018-01-11 20:30:00
Order Failed {
    "id": "e8be835d509a4fa886b757c8cb0d2c4e",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.05,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "FAILED",
    "created_time": "2018-01-19T20:32:39.516

---------------------------------------
Epoch 143 - Timestep: 2018-01-11T23:00:00
---------------------------------------
OHLCV
    O: 0.0869 | C: 0.0860 | V: 7064.4 | T: 2018-01-11T23:00:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0860 | Quantity: 0.0500
PERFORMANCE
    Cash: 1.0020 Total Val: 1.0020 PnL: 0.0020 Returns: 0.0020
BALANCE
    BTC - {'free': 1.0019986499999998, 'used': 0.0, 'total': 1.0019986499999998}
    ETH - {'free': 1.3877787807814457e-17, 'used': 0.0, 'total': 1.3877787807814457e-17}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 1.3877787807814457e-17, 'cost_price': 0.08798899999999998, 'latest_price': 0.086864}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 144 - Timestep: 2018-01-11T23:30:00
---------------------------------------
OHLCV
    O: 0.0860 | C: 0.0860 | V: 5216.2 | T: 2018-01-11T23:30:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 0.0500
PERFORMANCE
    Cash: 0.9977 Total Val: 1.0020 PnL: 0.0

2018-01-11 23:00:00
2018-01-11 23:00:00
Order Failed {
    "id": "4b77b5fa700c4662aa33ec3dc83e2699",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.05,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "FAILED",
    "created_time": "2018-01-19T20:32:39.730571",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "attempts": 1,
    "trades": [],
    "error": {
        "code": "INSUFFICIENT_FUNDS",
        "msg": "Insufficient funds to place order for asset ETH/BTC of cost: 0.00430. BTC balance is: 1.0019986499999998. ETH balance is 1.3877787807814457e-17"
    },
    "symbol": "ETH/BTC"
}
Equal? False
2018-01-11 23:30:00
2018-01-11 23:30:00
2018-01-11 23:30:00
Order Failed {
    "id": "6da435b88b224337a4fc489c0b5d89b8",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.0,
    "quantity": 0.05,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SE

## Simulate Orders

In [ ]:
cash = 1.0
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
exchange = load_ccxt_based_paper_exchange(balance, c.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)

asset = Asset(c.ETH, c.BTC)
feed = OHLCVExchangeFeed(
    exchange=exchange,
    assets=[asset],
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy(asset, quantity=.05)
experiment_name = 'default_simulate'

In [ ]:
runner.simulate(experiment_name, exchange, balance, portfolio, feed, strategy)

## Live Trade

In [ ]:
exchange = load_exchange(c.BINANCE)
cash = exchange.fetch_balance().get(c.BTC)[BalanceType.FREE]
print(c.BTC, "cash", cash)

In [ ]:
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)

In [ ]:
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
asset = Asset(c.ETH, c.BTC)
feed = OHLCVExchangeFeed(
    exchange=exchange,
    assets=[asset],
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy(asset, quantity=.05)
experiment_name = 'default_live'

In [ ]:
runner.live(experiment_name, exchange, balance, portfolio, feed, strategy)